# Final Review: Major Pneumonia Detection

This notebook contains the code for training and evaluating a pneumonia detection model using deep learning techniques. It includes data preprocessing, model architecture, training, and visualization of results.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
import cv2
from lime import lime_image
import time
from skimage.segmentation import mark_boundaries
import shap
from skimage.color import gray2rgb, rgb2gray, label2rgb


In [ ]:
# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_images = r"C:\Users\lucky\Downloads\archive\chest_xray\train"
train_generator = train_datagen.flow_from_directory(train_images, target_size=(300, 300), batch_size=128, class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(r"C:\Users\lucky\Downloads\archive\chest_xray\val", target_size=(300, 300), batch_size=128, class_mode='binary')


In [ ]:
# Model Architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Model Training
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)


In [ ]:
# Model Evaluation
eval_datagen = ImageDataGenerator(rescale=1/255)
test_generator = eval_datagen.flow_from_directory(r"C:\Users\lucky\Downloads\archive\chest_xray\test", target_size=(300, 300), batch_size=128, class_mode='binary')
eval_result = model.evaluate(test_generator, steps=624)
print('loss :', eval_result[0])
print('accuracy :', eval_result[1])


In [ ]:
# LIME Explanation
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(validation_generator.__getitem__(0)[0][11].astype('double'), model.predict, top_labels=1, hide_color=0, num_samples=1000)
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp, mask))


In [ ]:
# SHAP Explanation
background = x_train[np.random.choice(x_train.shape[0], 20, replace=False)]
e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(x_test[1:3])
shap.image_plot(shap_values, x_test[1:3])
